In [131]:
from google.colab import files
# 
uploaded=files.upload()
for fn in uploaded.keys():
    print('user uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))



Saving arjun.jpeg to arjun.jpeg
user uploaded file "arjun.jpeg" with length 60249 bytes


In [5]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import model_from_json

Using TensorFlow backend.


In [54]:
!ls


datalab		   model.hsamosa  WhatsApp Image 2018-08-08 at 12.48.20 AM.jpeg
frame15n (3).jpeg  model.json	  wwe
image.zip	   sample_data


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('image.zip', 'r')
zip_ref.extractall()
zip_ref.close()


In [8]:
from scipy.misc import imread
from scipy.misc import imresize
import os
ls=[]
folder='wwe'
files=os.listdir(folder)
files=list(map(lambda x: os.path.join(folder,x),files))
a=(len(files))
for i in range(a):
    im = imread(files[i])
    im=imresize(im,[32,32,3])
    im=im.flatten()
    ls.append(im)


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [9]:
import numpy as np

raw_data=np.array(ls,dtype=object)
print(raw_data.shape)

(2370, 3072)


In [10]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(raw_data,train_size=0.7,test_size=0.3,shuffle=True)
print(train_data.shape)

(1659, 3072)


In [11]:
(x_train, y_train), (x_test, y_test)=cifar10.load_data()

170500096/170498071 [==============================] - 22s 0us/step


In [97]:
x1=x_train[0:5000,:]
x2=x_train[5000:10000,:]
x1.shape,x2.shape

((5000, 32, 32, 3), (5000, 32, 32, 3))

In [98]:
x1=x1.reshape(5000,3072)
x2=x2.reshape(5000,3072)
x1.shape,x2.shape

((5000, 3072), (5000, 3072))

In [99]:
y1=y_train[0:5000]
y2=y_train[5000:10000]
y1.shape,y2.shape


((5000, 1), (5000, 1))

In [102]:
beg=5000
end=len(x1) + len(train_data)
#print(end)
images=np.zeros(shape=[end+1,3072])
images[0:beg, :]=x1
images[beg:end,:]=train_data
#images[end]=images[11330]
print(images.shape)

(6660, 3072)


In [0]:
images=images.reshape(-1,32,32,3)
images=images/255.0
#print(images[2])

In [104]:
images.shape

(6660, 32, 32, 3)

In [105]:
tes=x2[0:5000,:]
print(tes.shape)
end=len(tes) + len(test_data)
test_images=np.zeros(shape=[end,3072])
beg=5000
test_images[0:beg,:]=tes
test_images[beg:end,:]=test_data
print(test_images[5201])
print(test_images.shape)

(5000, 3072)
[254. 255. 255. ... 255. 253. 255.]
(5711, 3072)


In [106]:
test_images=test_images.reshape(-1,32,32,3)
print(test_images.shape)
test_images=test_images/255.0

(5711, 32, 32, 3)


In [107]:
test_images.shape

(5711, 32, 32, 3)

In [109]:
label_train=np.zeros(shape=[6660,1])
label_train[5000:6660,:]=1
print(label_train.shape)

(6660, 1)


In [110]:
label_test=np.zeros(shape=[5711,1])
label_test[5000:5711,:]=1
print(label_test[5001])

[1.]


In [111]:
a=(images.shape[0])
i=np.random.permutation(a)
x_train1,y_train1=images[i],label_train[i]
print(y_train1.shape,x_train1.shape)

(6660, 1) (6660, 32, 32, 3)


In [112]:
j=np.random.permutation(test_images.shape[0])
data_test,label_test=test_images[j],label_test[j]
print(data_test.shape,label_test.shape)

(5711, 32, 32, 3) (5711, 1)


In [0]:
num_classes=2
y_train_main = keras.utils.to_categorical(y_train1, num_classes)
y_test_main = keras.utils.to_categorical(label_test, num_classes)


In [114]:
y_train_main.shape,y_test_main.shape

((6660, 2), (5711, 2))

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train1.shape[1:]))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [0]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [117]:
batch_size=32
epochs=12
model.fit(x_train1, y_train_main,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(data_test, y_test_main),
              shuffle=True)

Train on 6660 samples, validate on 5711 samples
Epoch 1/12
6660/6660 [==============================] - 6s 965us/step - loss: 0.2288 - acc: 0.9084 - val_loss: 0.0724 - val_acc: 0.9713
Epoch 2/12
6660/6660 [==============================] - 5s 794us/step - loss: 0.0874 - acc: 0.9653 - val_loss: 0.1231 - val_acc: 0.9468
Epoch 3/12
6660/6660 [==============================] - 5s 797us/step - loss: 0.0702 - acc: 0.9736 - val_loss: 0.0540 - val_acc: 0.9809
Epoch 4/12
6660/6660 [==============================] - 5s 824us/step - loss: 0.0618 - acc: 0.9769 - val_loss: 0.0430 - val_acc: 0.9835
Epoch 5/12
6660/6660 [==============================] - 5s 805us/step - loss: 0.0550 - acc: 0.9787 - val_loss: 0.0889 - val_acc: 0.9652
Epoch 6/12
6660/6660 [==============================] - 5s 817us/step - loss: 0.0494 - acc: 0.9817 - val_loss: 0.0346 - val_acc: 0.9874
Epoch 7/12
6660/6660 [==============================] - 5s 820us/step - loss: 0.0470 - acc: 0.9808 - val_loss: 0.0332 - val_acc: 0.9877


In [118]:
scores = model.evaluate(data_test, y_test_main, verbose=1)
scores


5711/5711 [==============================] - 1s 244us/step


[0.04684034816154426, 0.984941341271231]

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [120]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [122]:
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [145]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.predict_classes(im, verbose=1)

1/1 [==============================] - 0s 460ms/step


In [146]:

print(score)

#if(score[0][0]>score[0][1]):
 # print("3")
#if(score[4][0]<score[4][1]):
 # print("3")
#for i in range(5711):
  #if(score[i][0]<score[i][0]):
  #print(score[i][1])
    #print(i)

[1]


In [144]:
from scipy.misc import imread
from scipy.misc import imresize
im = imread("frame15n (3).jpeg")
im=imresize(im,[32,32,3])
im=im/255.
im=im.reshape((1,)+im.shape)
print(im.shape)



(1, 32, 32, 3)


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [65]:
!ls

datalab
frame15n (3).jpeg
image.zip
model.hsamosa
model.json
phone2.jpeg
phone.jpeg
sample_data
WhatsApp Image 2018-08-08 at 12.48.20 AM.jpeg
WhatsApp Image 2018-08-22 at 9.02.51 PM (2).jpeg
WhatsApp Image 2018-08-22 at 9.02.51 PM (3).jpeg
WhatsApp Image 2018-08-22 at 9.02.52 PM (1).jpeg
WhatsApp Image 2018-08-22 at 9.02.52 PM (2).jpeg
WhatsApp Image 2018-08-22 at 9.02.52 PM (3).jpeg
WhatsApp Image 2018-08-22 at 9.02.52 PM (4).jpeg
WhatsApp Image 2018-08-22 at 9.02.52 PM (5).jpeg
WhatsApp Image 2018-08-22 at 9.02.52 PM (6).jpeg
WhatsApp Image 2018-08-22 at 9.02.52 PM.jpeg
wwe


In [0]:
from google.colab import files

In [0]:
files.download("model.h5")

In [0]:
files.download("model.json")